In [1]:
import torch
import torch.nn as nn

In [2]:
lenet = nn.Sequential(nn.Conv2d(1,6, kernel_size=5), nn.BatchNorm2d(6), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Conv2d(6,16, kernel_size=5), nn.BatchNorm2d(16), nn.ReLU(),nn.MaxPool2d(kernel_size=2,stride=2),
                      nn.Flatten(), 
                      nn.Linear(256, 120), nn.BatchNorm1d(120), nn.ReLU(),
                      nn.Linear(120,64), nn.BatchNorm1d(64), nn.ReLU(),
                      nn.Linear(64,10))

In [3]:
X = torch.randn(10,1,28,28)
def look_in_net(net, X):
    out = X
    for layer in net:
        out = layer(out)
        print(f"{layer.__class__.__name__} : {out.shape}")

look_in_net(lenet, X)

Conv2d : torch.Size([10, 6, 24, 24])
BatchNorm2d : torch.Size([10, 6, 24, 24])
ReLU : torch.Size([10, 6, 24, 24])
MaxPool2d : torch.Size([10, 6, 12, 12])
Conv2d : torch.Size([10, 16, 8, 8])
BatchNorm2d : torch.Size([10, 16, 8, 8])
ReLU : torch.Size([10, 16, 8, 8])
MaxPool2d : torch.Size([10, 16, 4, 4])
Flatten : torch.Size([10, 256])
Linear : torch.Size([10, 120])
BatchNorm1d : torch.Size([10, 120])
ReLU : torch.Size([10, 120])
Linear : torch.Size([10, 64])
BatchNorm1d : torch.Size([10, 64])
ReLU : torch.Size([10, 64])
Linear : torch.Size([10, 10])


z:\installs\anconda\envs\myenv\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [4]:
import torchvision

my_transforms = torchvision.transforms.Compose(
                [
                    torchvision.transforms.ToTensor(),
                ]
)

In [5]:
#train_dataset = torchvision.datasets.Cityscapes( root="../../data", transform=my_transforms)
#test_dataset = torchvision.datasets.Cityscapes(root="../../data", transform=my_transforms)
from torch.utils.data import DataLoader
from torchvision import datasets

batch_size=64

train_dataset = datasets.FashionMNIST(download=False, root="../../data", train=True, transform=my_transforms)
test_dataset = datasets.FashionMNIST(download=False, root="../../data", train=False, transform=my_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [6]:
for X,y in train_dataloader:
    print(X.shape, len(y))
    break

torch.Size([64, 1, 28, 28]) 64


In [7]:
net = lenet
loss_criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1)

In [8]:
num_epochs = 10

# def accuracy(y_hat, y):

#     return (torch.argmax(y_hat, dim=1)==y).sum().float().mean()

def accuracy(y_hat, y):
    return (torch.argmax(y_hat, dim=1)==y).sum().item()/len(y)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
net = net.to(device)
for epoch in range(num_epochs):
    for X, y in train_dataloader:
        X = X.to(device)
        y = y.to(device)
        
        y_hat = net(X)
        
        loss = loss_criterion(y_hat,y)
        current_acc = accuracy(y_hat, y)
        
        #print(current_acc.item())
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

tensor(9., device='cuda:0')
tensor(22., device='cuda:0')
tensor(27., device='cuda:0')
tensor(30., device='cuda:0')
tensor(30., device='cuda:0')
tensor(43., device='cuda:0')
tensor(38., device='cuda:0')
tensor(46., device='cuda:0')
tensor(42., device='cuda:0')
tensor(44., device='cuda:0')
tensor(41., device='cuda:0')
tensor(43., device='cuda:0')
tensor(41., device='cuda:0')
tensor(53., device='cuda:0')
tensor(40., device='cuda:0')
tensor(45., device='cuda:0')
tensor(48., device='cuda:0')
tensor(49., device='cuda:0')
tensor(37., device='cuda:0')
tensor(47., device='cuda:0')
tensor(50., device='cuda:0')
tensor(41., device='cuda:0')
tensor(46., device='cuda:0')
tensor(54., device='cuda:0')
tensor(54., device='cuda:0')
tensor(45., device='cuda:0')
tensor(49., device='cuda:0')
tensor(53., device='cuda:0')
tensor(44., device='cuda:0')
tensor(52., device='cuda:0')
tensor(51., device='cuda:0')
tensor(45., device='cuda:0')
tensor(44., device='cuda:0')
tensor(55., device='cuda:0')
tensor(54., dev

tensor(53., device='cuda:0')
tensor(52., device='cuda:0')
tensor(57., device='cuda:0')
tensor(56., device='cuda:0')
tensor(52., device='cuda:0')
tensor(48., device='cuda:0')
tensor(48., device='cuda:0')
tensor(52., device='cuda:0')
tensor(55., device='cuda:0')
tensor(58., device='cuda:0')
tensor(54., device='cuda:0')
tensor(51., device='cuda:0')
tensor(50., device='cuda:0')
tensor(56., device='cuda:0')
tensor(56., device='cuda:0')
tensor(53., device='cuda:0')
tensor(48., device='cuda:0')
tensor(56., device='cuda:0')
tensor(54., device='cuda:0')
tensor(57., device='cuda:0')
tensor(58., device='cuda:0')
tensor(49., device='cuda:0')
tensor(57., device='cuda:0')
tensor(53., device='cuda:0')
tensor(57., device='cuda:0')
tensor(56., device='cuda:0')
tensor(51., device='cuda:0')
tensor(57., device='cuda:0')
tensor(57., device='cuda:0')
tensor(56., device='cuda:0')
tensor(57., device='cuda:0')
tensor(55., device='cuda:0')
tensor(55., device='cuda:0')
tensor(53., device='cuda:0')
tensor(55., de

tensor(57., device='cuda:0')
tensor(52., device='cuda:0')
tensor(56., device='cuda:0')
tensor(49., device='cuda:0')
tensor(58., device='cuda:0')
tensor(62., device='cuda:0')
tensor(54., device='cuda:0')
tensor(59., device='cuda:0')
tensor(55., device='cuda:0')
tensor(57., device='cuda:0')
tensor(58., device='cuda:0')
tensor(55., device='cuda:0')
tensor(56., device='cuda:0')
tensor(51., device='cuda:0')
tensor(57., device='cuda:0')
tensor(55., device='cuda:0')
tensor(53., device='cuda:0')
tensor(53., device='cuda:0')
tensor(58., device='cuda:0')
tensor(55., device='cuda:0')
tensor(57., device='cuda:0')
tensor(54., device='cuda:0')
tensor(57., device='cuda:0')
tensor(61., device='cuda:0')
tensor(56., device='cuda:0')
tensor(54., device='cuda:0')
tensor(55., device='cuda:0')
tensor(53., device='cuda:0')
tensor(51., device='cuda:0')
tensor(62., device='cuda:0')
tensor(57., device='cuda:0')
tensor(53., device='cuda:0')
tensor(57., device='cuda:0')
tensor(54., device='cuda:0')
tensor(51., de

tensor(54., device='cuda:0')
tensor(53., device='cuda:0')
tensor(53., device='cuda:0')
tensor(58., device='cuda:0')
tensor(54., device='cuda:0')
tensor(60., device='cuda:0')
tensor(58., device='cuda:0')
tensor(56., device='cuda:0')
tensor(57., device='cuda:0')
tensor(61., device='cuda:0')
tensor(57., device='cuda:0')
tensor(54., device='cuda:0')
tensor(50., device='cuda:0')
tensor(56., device='cuda:0')
tensor(56., device='cuda:0')
tensor(58., device='cuda:0')
tensor(57., device='cuda:0')
tensor(56., device='cuda:0')
tensor(55., device='cuda:0')
tensor(56., device='cuda:0')
tensor(55., device='cuda:0')
tensor(56., device='cuda:0')
tensor(56., device='cuda:0')
tensor(57., device='cuda:0')
tensor(56., device='cuda:0')
tensor(55., device='cuda:0')
tensor(56., device='cuda:0')
tensor(59., device='cuda:0')
tensor(58., device='cuda:0')
tensor(56., device='cuda:0')
tensor(58., device='cuda:0')
tensor(56., device='cuda:0')
tensor(53., device='cuda:0')
tensor(60., device='cuda:0')
tensor(50., de

tensor(57., device='cuda:0')
tensor(59., device='cuda:0')
tensor(55., device='cuda:0')
tensor(54., device='cuda:0')
tensor(58., device='cuda:0')
tensor(61., device='cuda:0')
tensor(53., device='cuda:0')
tensor(55., device='cuda:0')
tensor(53., device='cuda:0')
tensor(56., device='cuda:0')
tensor(59., device='cuda:0')
tensor(56., device='cuda:0')
tensor(56., device='cuda:0')
tensor(52., device='cuda:0')
tensor(57., device='cuda:0')
tensor(56., device='cuda:0')
tensor(59., device='cuda:0')
tensor(59., device='cuda:0')
tensor(58., device='cuda:0')
tensor(60., device='cuda:0')
tensor(56., device='cuda:0')
tensor(56., device='cuda:0')
tensor(58., device='cuda:0')
tensor(58., device='cuda:0')
tensor(56., device='cuda:0')
tensor(55., device='cuda:0')
tensor(57., device='cuda:0')
tensor(54., device='cuda:0')
tensor(57., device='cuda:0')
tensor(56., device='cuda:0')
tensor(59., device='cuda:0')
tensor(57., device='cuda:0')
tensor(60., device='cuda:0')
tensor(58., device='cuda:0')
tensor(57., de

...

tensor(60., device='cuda:0')
tensor(64., device='cuda:0')
tensor(56., device='cuda:0')
tensor(59., device='cuda:0')
tensor(58., device='cuda:0')
tensor(58., device='cuda:0')
tensor(59., device='cuda:0')
tensor(61., device='cuda:0')
tensor(57., device='cuda:0')
tensor(57., device='cuda:0')
tensor(60., device='cuda:0')
tensor(58., device='cuda:0')
tensor(60., device='cuda:0')
tensor(58., device='cuda:0')
tensor(58., device='cuda:0')
tensor(62., device='cuda:0')
tensor(61., device='cuda:0')
tensor(59., device='cuda:0')
tensor(61., device='cuda:0')
tensor(55., device='cuda:0')
tensor(61., device='cuda:0')
tensor(60., device='cuda:0')
tensor(59., device='cuda:0')
tensor(58., device='cuda:0')
tensor(58., device='cuda:0')
tensor(59., device='cuda:0')
tensor(58., device='cuda:0')
tensor(61., device='cuda:0')
tensor(56., device='cuda:0')
tensor(59., device='cuda:0')
tensor(62., device='cuda:0')
tensor(56., device='cuda:0')
tensor(59., device='cuda:0')
tensor(58., device='cuda:0')
tensor(58., de

tensor(60., device='cuda:0')
tensor(59., device='cuda:0')
tensor(60., device='cuda:0')
tensor(57., device='cuda:0')
tensor(61., device='cuda:0')
tensor(60., device='cuda:0')
tensor(59., device='cuda:0')
tensor(63., device='cuda:0')
tensor(56., device='cuda:0')
tensor(59., device='cuda:0')
tensor(62., device='cuda:0')
tensor(57., device='cuda:0')
tensor(56., device='cuda:0')
tensor(60., device='cuda:0')
tensor(59., device='cuda:0')
tensor(54., device='cuda:0')
tensor(61., device='cuda:0')
tensor(61., device='cuda:0')
tensor(55., device='cuda:0')
tensor(61., device='cuda:0')
tensor(60., device='cuda:0')
tensor(60., device='cuda:0')
tensor(58., device='cuda:0')
tensor(57., device='cuda:0')
tensor(62., device='cuda:0')
tensor(58., device='cuda:0')
tensor(62., device='cuda:0')
tensor(62., device='cuda:0')
tensor(61., device='cuda:0')
tensor(60., device='cuda:0')
tensor(59., device='cuda:0')
tensor(57., device='cuda:0')
tensor(58., device='cuda:0')
tensor(58., device='cuda:0')
tensor(61., de

tensor(60., device='cuda:0')
tensor(58., device='cuda:0')
tensor(61., device='cuda:0')
tensor(60., device='cuda:0')
tensor(58., device='cuda:0')
tensor(57., device='cuda:0')
tensor(55., device='cuda:0')
tensor(61., device='cuda:0')
tensor(55., device='cuda:0')
tensor(53., device='cuda:0')
tensor(60., device='cuda:0')
tensor(61., device='cuda:0')
tensor(59., device='cuda:0')
tensor(58., device='cuda:0')
tensor(59., device='cuda:0')
tensor(61., device='cuda:0')
tensor(58., device='cuda:0')
tensor(57., device='cuda:0')
tensor(63., device='cuda:0')
tensor(61., device='cuda:0')
tensor(60., device='cuda:0')
tensor(53., device='cuda:0')
tensor(58., device='cuda:0')
tensor(60., device='cuda:0')
tensor(57., device='cuda:0')
tensor(62., device='cuda:0')
tensor(59., device='cuda:0')
tensor(63., device='cuda:0')
tensor(58., device='cuda:0')
tensor(56., device='cuda:0')
tensor(57., device='cuda:0')
tensor(61., device='cuda:0')
tensor(60., device='cuda:0')
tensor(62., device='cuda:0')
tensor(59., de

tensor(61., device='cuda:0')
tensor(59., device='cuda:0')
tensor(56., device='cuda:0')
tensor(58., device='cuda:0')
tensor(58., device='cuda:0')
tensor(60., device='cuda:0')
tensor(60., device='cuda:0')
tensor(57., device='cuda:0')
tensor(60., device='cuda:0')
tensor(61., device='cuda:0')
tensor(60., device='cuda:0')
tensor(60., device='cuda:0')
tensor(61., device='cuda:0')
tensor(62., device='cuda:0')
tensor(59., device='cuda:0')
tensor(62., device='cuda:0')
tensor(56., device='cuda:0')
tensor(57., device='cuda:0')
tensor(59., device='cuda:0')
tensor(60., device='cuda:0')
tensor(61., device='cuda:0')
tensor(62., device='cuda:0')
tensor(59., device='cuda:0')
tensor(58., device='cuda:0')
tensor(62., device='cuda:0')
tensor(59., device='cuda:0')
tensor(60., device='cuda:0')
tensor(61., device='cuda:0')
tensor(57., device='cuda:0')
tensor(61., device='cuda:0')
tensor(54., device='cuda:0')
tensor(62., device='cuda:0')
tensor(60., device='cuda:0')
tensor(54., device='cuda:0')
tensor(50., de

tensor(59., device='cuda:0')
tensor(59., device='cuda:0')
tensor(59., device='cuda:0')
tensor(63., device='cuda:0')
tensor(59., device='cuda:0')
tensor(61., device='cuda:0')
tensor(58., device='cuda:0')
tensor(62., device='cuda:0')
tensor(62., device='cuda:0')
tensor(61., device='cuda:0')
tensor(61., device='cuda:0')
tensor(57., device='cuda:0')
tensor(57., device='cuda:0')
tensor(59., device='cuda:0')
tensor(58., device='cuda:0')
tensor(61., device='cuda:0')
tensor(59., device='cuda:0')
tensor(63., device='cuda:0')
tensor(59., device='cuda:0')
tensor(59., device='cuda:0')
tensor(56., device='cuda:0')
tensor(57., device='cuda:0')
tensor(60., device='cuda:0')
tensor(61., device='cuda:0')
tensor(57., device='cuda:0')
tensor(55., device='cuda:0')
tensor(60., device='cuda:0')
tensor(60., device='cuda:0')
tensor(57., device='cuda:0')
tensor(58., device='cuda:0')
tensor(61., device='cuda:0')
tensor(61., device='cuda:0')
tensor(58., device='cuda:0')
tensor(54., device='cuda:0')
tensor(57., de

In [12]:
y_hat = net(X)

accuracy(y_hat, y)

tensor(31., device='cuda:0')

In [16]:
y_hat.shape

torch.Size([32, 10])

In [17]:
y_hat[:2]

tensor([[-2.8773, -1.8495, -2.3771, -2.3750,  1.3213, -0.2911, -1.8647, -2.3899,
         15.6756, -4.1942],
        [-1.6005, -2.1848, -1.5011,  0.2045, -2.9402,  0.4956, -1.1805,  6.1580,
          0.3053,  3.2958]], device='cuda:0', grad_fn=<SliceBackward>)

In [20]:
torch.argmax(y_hat, dim=1).shape

torch.Size([32])

In [25]:
torch.argmax(y_hat, dim=1)==y

# this is a high accuracy

tensor([ True,  True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True], device='cuda:0')

In [29]:
(torch.argmax(y_hat, dim=1)==y).sum().item()/len(y)

0.96875

In [30]:
# so our accuracy should actually bedefined as 

def accuracy(y_hat, y):
    return (torch.argmax(y_hat, dim=1)==y).sum().item()/len(y)